In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [3]:
subtype_labels = {
    'no-damage': 0,
    'minor-damage': 1,
    'major-damage': 2,
    'destroyed': 3
}


In [4]:
# Load CSV metadata
metadata_df = pd.read_csv('building_polygons_metadata.csv')

# Filter for post-disaster images with a valid subtype
post_disaster_df = metadata_df.loc[
    (metadata_df['stage'] == 'post') & 
    (metadata_df['subtype'].isin(subtype_labels))
].copy()

# Assign integer label for subtype
post_disaster_df['subtype_label'] = post_disaster_df['subtype'].map(subtype_labels)

# Build image path
image_dir = 'cropped_square_buildings'
post_disaster_df['image_path'] = post_disaster_df.apply(
    lambda row: os.path.join(image_dir, f"{row['uid']}_{row['stage']}.png"), axis=1
)

# Keep only rows where the image file exists
post_disaster_df = post_disaster_df[post_disaster_df['image_path'].apply(os.path.isfile)]

print(f"Total images with subtype: {len(post_disaster_df)}")
print(post_disaster_df['subtype'].value_counts())


Total images with subtype: 115417
subtype
no-damage       83824
minor-damage    11543
major-damage    10071
destroyed        9979
Name: count, dtype: int64


In [5]:
train_df, temp_df = train_test_split(
    post_disaster_df, test_size=0.3, random_state=42, stratify=post_disaster_df['subtype_label']
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df['subtype_label']
)
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")


Train: 80791, Val: 17313, Test: 17313


In [6]:
class SubtypeImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = Image.open(row['image_path']).convert('RGB')
        label = row['subtype_label']
        if self.transform:
            image = self.transform(image)
        return image, label


In [7]:
img_size = 224
train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = SubtypeImageDataset(train_df, transform=train_transform)
val_dataset = SubtypeImageDataset(val_df, transform=val_test_transform)
test_dataset = SubtypeImageDataset(test_df, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = models.resnet50(weights='IMAGENET1K_V1')
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4 subtypes
model = model.to(device)


Using device: cuda


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, save_path='best_subtype_model.pth'):
    best_val_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    epoch_pbar = tqdm(range(num_epochs), desc="Training Progress", unit="epoch", leave=True)

    for epoch in epoch_pbar:
        model.train()
        running_loss = 0.0
        running_corrects = 0
        batch_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Training]', leave=False, unit="batch")
        for inputs, labels in batch_pbar:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            batch_pbar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'acc': f"{torch.sum(preds == labels.data).item()/inputs.size(0):.2%}"
            })
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc.item())

        # Validation
        model.eval()
        val_running_loss = 0.0
        val_running_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * inputs.size(0)
                val_running_corrects += torch.sum(preds == labels.data)
        val_epoch_loss = val_running_loss / len(val_loader.dataset)
        val_epoch_acc = val_running_corrects.double() / len(val_loader.dataset)
        history['val_loss'].append(val_epoch_loss)
        history['val_acc'].append(val_epoch_acc.item())
        epoch_pbar.set_postfix({
            'train_loss': f"{epoch_loss:.4f}",
            'train_acc': f"{epoch_acc:.2%}",
            'val_loss': f"{val_epoch_loss:.4f}",
            'val_acc': f"{val_epoch_acc:.2%}"
        })
        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            try:
                torch.save(model.state_dict(), save_path)
                tqdm.write(f"Model saved to {save_path} at epoch {epoch+1}")
            except Exception as e:
                tqdm.write(f"Warning: Could not save model at epoch {epoch+1}: {e}")

    tqdm.write("Training complete.")
    return model, history

# Train
model, history = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=15)


In [ ]:
# Load best model
model.load_state_dict(torch.load('best_subtype_model.pth'))

def evaluate_model(model, test_loader):
    model.eval()
    test_corrects = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating on test set", unit="batch"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            test_corrects += torch.sum(preds == labels.data)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    test_acc = test_corrects.double() / len(test_loader.dataset)
    print(f'Test Accuracy: {test_acc:.4f}')
    return test_acc, all_preds, all_labels

test_accuracy, predictions, true_labels = evaluate_model(model, test_loader)


In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train')
plt.plot(history['val_loss'], label='Validation')
plt.title('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='Train')
plt.plot(history['val_acc'], label='Validation')
plt.title('Accuracy')
plt.legend()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

class_names = list(subtype_labels.keys())
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
